In [1]:
import os
import sys
import pandas as pd
import numpy as np
from rpy2.robjects import packages, pandas2ri, r
import dask.dataframe as dd

PROJECT_PATH = os.path.abspath(os.path.join(os.getcwd(), '../'))

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)
    
from src.data.fitzroy_data import fitzroy, r_to_pandas

In [2]:
%%time

# Player data matches have weird round labelling system (lots of strings for finals matches),
# so using round numbers from match_results
match_df = dd.from_pandas(r_to_pandas(r('fitzRoy::match_results')), chunksize=5000)
player_df = (dd.from_pandas(r_to_pandas(fitzroy().get_afltables_stats()), chunksize=5000)
             # Some player data venues have trailing spaces
             .assign(venue=lambda x: x['venue'].str.strip())
             # Player data match IDs are wrong for recent years. The easiest way to add correct ones
             # is to graft on the IDs from match_results. Also, match_results round_numbers are more useful.
             .merge(match_df[['date', 'venue', 'round_number', 'game']], on=['date', 'venue'], how='left')
             # As of 11-10-2018, match_results is still missing finals data from 2018.
             # Joining on date/venue leaves two duplicates played at M.C.G. on 29-4-1986 & 9-8-1986,
             # but that's an acceptable loss of data and easier than munging team names
             .dropna()
             .rename(columns={'season': 'year',
                              'time_on_ground__': 'time_on_ground',
                              'id': 'player_id',
                              'game': 'match_id'})
             .astype({'year': int, 'match_id': int})
             .assign(player_name=lambda x: x['first_name'] + ' ' + x['surname'],
                     # Need to add year to ID, because there are some player_id/match_id combos, decades apart,
                     # that by chance overlap
                     id=lambda x: x['player_id'].astype(str) + x['match_id'].astype(str) + x['year'].astype(str))
             .drop(['first_name', 'surname', 'round', 'local_start_time', 'attendance', 'hq1g', 'hq1b',
                    'hq2g', 'hq2b', 'hq3g', 'hq3b', 'hq4g', 'hq4b', 'aq1g', 'aq1b', 'aq2g', 'aq2b', 'aq3g', 'aq3b',
                    'aq4g', 'aq4b', 'jumper_no_', 'umpire_1', 'umpire_2', 'umpire_3', 'umpire_4',
                    'substitute', 'group_id', 'date', 'venue'], axis=1)
             # Some early matches (1800s) have fully-duplicated rows
             .drop_duplicates()
             .set_index('id', drop=False))

stats_cols = ['player_id', 'kicks', 'marks', 'handballs', 'goals', 'behinds', 'hit_outs', 'tackles',
               'rebounds', 'inside_50s', 'clearances', 'clangers', 'frees_for',
               'frees_against', 'contested_possessions',
               'uncontested_possessions', 'contested_marks', 'marks_inside_50',
               'one_percenters', 'bounces', 'goal_assists', 'time_on_ground']

player_groups = player_df[stats_cols].groupby('player_id')

rolling_stats = player_groups.apply(lambda x: x.rolling(23).mean(),
                                    meta={col: 'f8' if col != 'player_id' else 'int' for col in stats_cols})
expanding_stats = (player_groups
                   .apply(lambda x: x.expanding(1).mean(),
                          meta={col: 'f8' for col in stats_cols}))
blank_rows = rolling_stats.isna().any(axis=1)
npartitions = player_df.npartitions
expanding_rolling_stats = (dd.multi
                           .concat([rolling_stats, expanding_stats[blank_rows]])
                           .dropna()
                           .drop('player_id', axis=1))

brownlow_last_year = dd.from_pandas(player_df[['player_id', 'year', 'brownlow_votes']]
                      .groupby(['player_id', 'year'])
                      .sum()
                      .compute()
                      # Can't figure out how to get Dask to play nice with groupby + shift
                      .groupby('player_id')
                      .shift()
                      .reset_index(level=0)
                      .fillna(0)
                      .rename(columns={'brownlow_votes': 'last_year_brownlow_votes'}), chunksize=10000)

brownlow_df = (dd.multi.merge(player_df[['id', 'year', 'home_team', 'home_score', 'away_team', 'away_score',
                                        'player_id', 'playing_for', 'round_number', 'match_id', 'player_name']],
                             brownlow_last_year,
                             on=['player_id', 'year'], how='left')
                .set_index('id')
                .assign(cum_games_played=player_groups.cumcount()))

player_stats = dd.multi.concat([brownlow_df.repartition(npartitions=npartitions).reset_index(drop=True),
                                expanding_rolling_stats.repartition(npartitions=npartitions).reset_index(drop=True)],
                               axis=1)

home_stats = (player_stats
              [player_stats['playing_for'] == player_stats['home_team']]
              .drop(['playing_for'], axis=1)
              .rename(columns=lambda x: x.replace('home_', '').replace('away_', 'oppo_'))
              .assign(at_home=1))
away_stats = (player_stats
              [player_stats['playing_for'] == player_stats['away_team']]
              .drop(['playing_for'], axis=1)
              .rename(columns=lambda x: x.replace('away_', '').replace('home_', 'oppo_'))
              .assign(at_home=0))
 
match_cols = ['at_home', 'match_id', 'score', 'oppo_score']
player_aggs = {col: 'sum' for col in stats_cols[1:] + ['last_year_brownlow_votes']}
match_aggs = {col: 'mean' for col in match_cols}

aggregations = {**player_aggs, **match_aggs}

team_df = (dd.multi.concat([home_stats[home_stats.columns.sort_values()],
                           away_stats[away_stats.columns.sort_values()]],
                           interleave_partitions=True)
           .drop(['player_id', 'player_name'], axis=1)
           .groupby(['team', 'year', 'round_number', 'oppo_team'])
           .agg(aggregations)
           .reset_index()
           .drop_duplicates(subset=['team', 'year', 'round_number'])
           .compute())

team_df

/usr/local/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Returning data from 1897-01-01 to 2018-10-14

  warnings.warn(x, RRuntimeWarning)




|==                                                    |  4% ~3 s remaining

 

 

 

 

 


|===                                                   |  7% ~4 s remaining

 

 

 

 

 


/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Downloading data


  warnings.warn(x, RRuntimeWarning)




|====                                                  |  9% ~4 s remaining

 

 

 

 

 


|======                                                | 11% ~5 s remaining

 

 

 

 

 


|=======                                               | 13% ~5 s remaining

 

 

 

 

 


|========                                              | 16% ~5 s remaining

 

 

 

 

 


|=========                                             | 18% ~5 s remaining

 

 

 

 

 


|==========                                            | 20% ~5 s remaining

 

 

 

 

 


|============                                          | 22% ~5 s remaining

 

 

 

 

 


|=============                                         | 24% ~5 s remaining

 

 

 

 

 


|==============                                        | 27% ~5 s remaining

 

 

 

 

 


|===============                                       | 29% ~5 s remaining

 

 

 

 

 


|================                                      | 31% ~4 s re

/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Finished downloading data. Processing XMLs


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Warning:
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  396 parsing failures.
row # A tibble: 5 x 5 col     row col   expected   actual file                                             expected   <int> <chr> <chr>      <chr>  <chr>                                            actual 1  8713 Round an integer QF     'https://afltables.com/afl/stats/2018_stats.txt' file 2  8714 Round an integer QF     'https://afltables.com/afl/stats/2018_stats.txt' row 3  8715 Round an integer QF     'https://afltables.com/afl/stats/2018_stats.txt' col 4  8716 Round an integer QF     'https://afltables.com/afl/stats/2018_stats.txt' expected 5  8717 Round an integer QF     '

CPU times: user 5min 57s, sys: 1min 9s, total: 7min 6s
Wall time: 6min 30s


In [3]:
team_df

,team,year,round_number,oppo_team,kicks,marks,handballs,goals,behinds,hit_outs,...,marks_inside_50,one_percenters,bounces,goal_assists,time_on_ground,last_year_brownlow_votes,at_home,match_id,score,oppo_score
0,Adelaide,1991,1.0,Hawthorn,142.877470,61.739130,83.881423,15.026877,7.122530,1.308300,...,7.662055,16.393281,4.446640,4.507905,906.189723,14.0,1.0,10128.0,155.0,69.0
1,Adelaide,1991,2.0,Carlton,144.217391,62.652174,88.304348,10.573499,8.434783,2.086957,...,8.000000,15.434783,2.826087,3.304348,918.652174,14.0,1.0,10135.0,81.0,104.0
2,Adelaide,1991,3.0,Sydney,150.028986,61.478261,77.681159,17.322793,8.710145,2.869565,...,4.913043,12.130435,2.217391,2.753623,670.115942,27.0,0.0,10143.0,132.0,108.0
3,Adelaide,1991,4.0,Essendon,180.250000,68.967391,88.054348,16.543478,9.891304,2.304348,...,7.086957,15.826087,2.869565,3.586957,749.630435,27.0,0.0,10147.0,47.0,92.0
4,Adelaide,1991,5.0,West Coast,175.617391,67.956522,92.086957,14.156522,7.965217,1.521739,...,7.539130,18.508696,3.565217,3.486957,835.600000,14.0,0.0,10159.0,65.0,130.0
5,Adelaide,1991,6.0,Western Bulldogs,163.463768,60.521739,82.217391,11.695652,7.195652,1.043478,...,5.579710,16.347826,3.130435,2.927536,589.652174,27.0,1.0,10166.0,128.0,97.0
6,Adelaide,1991,7.0,St Kilda,145.431677,55.714286,79.111801,9.565217,5.732919,2.055901,...,6.509317,15.484472,3.304348,3.354037,665.043478,27.0,0.0,10168.0,31.0,162.0
7,Adelaide,1991,9.0,North Melbourne,141.657609,55.527174,73.760870,15.217391,7.451087,2.581522,...,8.771739,17.972826,3.521739,4.141304,744.233696,27.0,0.0,10181.0,118.0,120.0
8,Adelaide,1991,10.0,Melbourne,149.154589,58.149758,78.188406,15.565217,7.033816,2.579710,...,8.526570,17.599034,3.695652,4.057971,753.347826,27.0,1.0,10188.0,106.0,72.0
9,Adelaide,1991,11.0,Geelong,157.634783,60.313043,84.860870,15.839130,7.630435,2.569565,...,9.213043,17.934783,3.434783,4.469565,833.830435,27.0,0.0,10195.0,93.0,177.0
